In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
from scipy import stats
from IPython.core.interactiveshell import InteractiveShell

In [2]:
# Libraries Settings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=Warning)
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
np.set_printoptions(suppress=True)

def set_seed(seed=42):
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
pd.set_option("display.width", 100)
pd.set_option("display.max_columns", 60)
pd.set_option("display.max_rows", 25)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [3]:
df_deals = pd.read_excel ('data/Deals (investment).xlsx')
df_deals.head()

,1_post_date,2_post_title,3_amount,4_Stake,5_funding_round,6_investors,7_source,Country/Town,Country(HQ),Category,main_sector,Check vs Companies,"Deals Information, Level of Completeness",duplicated_conc,Duplication,Disclosed,Founded,Total Disclosed Funding,DA Classification_African Company (Yes = 0; No = 1),"funding round, DA","Industry, DA",Year,Month,Quarter,Half,Number of Investors,Investor 1,Investor 2,Investor 3,Investor 4,Investor 5,Investor 6,Investor 7,Investor 8,Investor 9,Investor 10,Investor 11,Investor 12,Investor 13,Investor 14,Investor 15,Cummulative Deals (Disclosed),Cummulative Deals (Undisclosed),Cummulative Amount,Investor 1.1,Investor 2.1,Investor 3.1,Investor 4.1,Investor 5.1,Investor 6.1,Investor 7.1,Investor 8.1,Investor 9.1,Investor 10.1,Investor 11.1,Investor 12.1,Investor 13.1,Investor 14.1,Investor 15.1
0,2008-09-01,biNu,600000,NaN,Seed,Artesian VC,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000,9.000,3.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-11-01,biNu,400000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,Australia,Australia,Mobile Internet,Information Technology,biNu,0.900,NaN,NaN,1.000,2008.000,14220000.000,1.000,2. Seed,Other Technologies & Information Technology,2008.000,11.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-12-03,AllLife,6000000,NaN,Private Equity,LeapFrog Investments,http://www.prnewswire.co.uk/news-releases/leap...,South Africa,South Africa,"Insurance, InsurTech",Financial Services,AllLife,0.900,NaN,NaN,1.000,2004.000,12700000.000,0.000,6. Private Equity,Financial Services,2009.000,12.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-12-11,Bridge International Academies,1800000,NaN,Grant,Omidyar Network,http://foundationcenter.org/pnd/news/story.jht...,NaN,NaN,NaN,NaN,NaN,0.643,NaN,NaN,1.000,NaN,27800000.000,NaN,1. Grant,NaN,2009.000,12.000,4.000,2.000,1.000,Omidyar Network,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,Omidyar Network,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-01,biNu,320000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.000,4.000,2.000,1.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(5))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df_deals)

##################### Shape #####################
(2059, 59)
##################### Types #####################
1_post_date        datetime64[ns]
2_post_title               object
3_amount                   object
4_Stake                   float64
5_funding_round            object
                        ...      
Investor 11.1              object
Investor 12.1              object
Investor 13.1              object
Investor 14.1              object
Investor 15.1             float64
Length: 59, dtype: object
##################### Head #####################
  1_post_date                    2_post_title 3_amount  4_Stake 5_funding_round  \
0  2008-09-01                            biNu   600000      NaN            Seed   
1  2008-11-01                            biNu   400000      NaN           Angel   
2  2009-12-03                         AllLife  6000000      NaN  Private Equity   
3  2009-12-11  Bridge International Academies  1800000      NaN           Grant   
4  2010-04-01            

In [5]:
cols = df_deals.columns

In [6]:
cols

Index(['1_post_date', '2_post_title', '3_amount', '4_Stake', '5_funding_round', '6_investors',
       '7_source', 'Country/Town', 'Country(HQ)', 'Category', 'main_sector', 'Check vs Companies',
       'Deals Information, Level of Completeness', 'duplicated_conc', 'Duplication', 'Disclosed ',
       'Founded', 'Total  Disclosed Funding',
       'DA Classification_African Company (Yes = 0; No = 1)', 'funding round, DA', 'Industry, DA',
       'Year', 'Month', 'Quarter', 'Half', 'Number of Investors', 'Investor 1', 'Investor 2',
       'Investor 3', 'Investor 4', 'Investor 5', 'Investor 6', 'Investor 7', 'Investor 8',
       'Investor 9', 'Investor 10', 'Investor 11', 'Investor 12', 'Investor 13', 'Investor 14',
       'Investor 15', 'Cummulative Deals (Disclosed)', 'Cummulative Deals (Undisclosed)',
       'Cummulative Amount', 'Investor 1.1', 'Investor 2.1', 'Investor 3.1', 'Investor 4.1',
       'Investor 5.1', 'Investor 6.1', 'Investor 7.1', 'Investor 8.1', 'Investor 9.1',
       'Inv

In [7]:
for cat_col in cols:
    print(f"{cat_col}: {df_deals[cat_col].nunique()} uniqueness variable")

1_post_date: 695 uniqueness variable
2_post_title: 800 uniqueness variable
3_amount: 281 uniqueness variable
4_Stake: 7 uniqueness variable
5_funding_round: 27 uniqueness variable
6_investors: 717 uniqueness variable
7_source: 1070 uniqueness variable
Country/Town: 34 uniqueness variable
Country(HQ): 34 uniqueness variable
Category: 660 uniqueness variable
main_sector: 48 uniqueness variable
Check vs Companies: 727 uniqueness variable
Deals Information, Level of Completeness: 6 uniqueness variable
duplicated_conc: 1120 uniqueness variable
Duplication: 1 uniqueness variable
Disclosed : 2 uniqueness variable
Founded: 23 uniqueness variable
Total  Disclosed Funding: 280 uniqueness variable
DA Classification_African Company (Yes = 0; No = 1): 2 uniqueness variable
funding round, DA: 8 uniqueness variable
Industry, DA: 13 uniqueness variable
Year: 14 uniqueness variable
Month: 12 uniqueness variable
Quarter: 4 uniqueness variable
Half: 2 uniqueness variable
Number of Investors: 12 uniquenes

In [8]:
df_deals = df_deals.drop(['Investor 13.1','Investor 14.1','Investor 15.1','Investor 15','Duplication'], axis=1)

In [9]:
df_deals.shape

(2059, 54)

In [10]:
df_deals.head(20)

,1_post_date,2_post_title,3_amount,4_Stake,5_funding_round,6_investors,7_source,Country/Town,Country(HQ),Category,main_sector,Check vs Companies,"Deals Information, Level of Completeness",duplicated_conc,Disclosed,Founded,Total Disclosed Funding,DA Classification_African Company (Yes = 0; No = 1),"funding round, DA","Industry, DA",Year,Month,Quarter,Half,Number of Investors,Investor 1,Investor 2,Investor 3,Investor 4,Investor 5,Investor 6,Investor 7,Investor 8,Investor 9,Investor 10,Investor 11,Investor 12,Investor 13,Investor 14,Cummulative Deals (Disclosed),Cummulative Deals (Undisclosed),Cummulative Amount,Investor 1.1,Investor 2.1,Investor 3.1,Investor 4.1,Investor 5.1,Investor 6.1,Investor 7.1,Investor 8.1,Investor 9.1,Investor 10.1,Investor 11.1,Investor 12.1
0,2008-09-01,biNu,600000,NaN,Seed,Artesian VC,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000,9.000,3.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-11-01,biNu,400000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,Australia,Australia,Mobile Internet,Information Technology,biNu,0.900,NaN,1.000,2008.000,14220000.000,1.000,2. Seed,Other Technologies & Information Technology,2008.000,11.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-12-03,AllLife,6000000,NaN,Private Equity,LeapFrog Investments,http://www.prnewswire.co.uk/news-releases/leap...,South Africa,South Africa,"Insurance, InsurTech",Financial Services,AllLife,0.900,NaN,1.000,2004.000,12700000.000,0.000,6. Private Equity,Financial Services,2009.000,12.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-12-11,Bridge International Academies,1800000,NaN,Grant,Omidyar Network,http://foundationcenter.org/pnd/news/story.jht...,NaN,NaN,NaN,NaN,NaN,0.643,NaN,1.000,NaN,27800000.000,NaN,1. Grant,NaN,2009.000,12.000,4.000,2.000,1.000,Omidyar Network,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Omidyar Network,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-01,biNu,320000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.000,4.000,2.000,1.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010-08-20,Kopo Kopo,75000,NaN,Seed,First Light Ventures,https://www.crunchbase.com/organization/kopo-k...,Kenya,Kenya,"Enterprise Software, Financial Services, Infor...",Financial Services,Kopo Kopo,0.900,40410Kopo Kopo75000,1.000,2011.000,6504900.000,0.000,2. Seed,Financial Services,2010.000,8.000,3.000,2.000,1.000,First Light Ventures,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,First Light Ventures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2010-09-30,Paga,700000,NaN,Seed,Tim Draper,https://www.crunchbase.com/funding_round/paga-...,Nigeria,Nigeria,"Money Transfer, Airtime, Bill Payments, and SM...",Financial Services,Paga,0.900,40451Paga700000,1.000,2009.000,32700000.000,0.000,2. Seed,Financial Services,2010.000,9.000,3.000,2.000,1.000,First Light Ventures,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,First Light Ventures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010-12-01,Wuzzuf,150000,NaN,Grant,Undisclosed,https://www.crunchbase.com/funding_round/wuzzu...,Egypt,Egypt,"Information Services, Information Technology, ...",Recruitment,Wuzzuf,0.900,40513Wuzzuf150000,1.000,2009.000,7850000.000,0.000,1. Grant,"Employment, Talent & Labour",2010.000,12.000,4.000,2.000,1.000,First Light Ventures,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,First Light Ventures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2011-05-01,African Leadership Academy,1500000,NaN,Grant,Omidyar Network,https://www.crunchbase.com/funding_round/afric...,South Africa,South Africa,"E-Learning, Education, Tutoring",Education,African Leadership Academy,0.900

In [11]:
#find duplicate rows across all columns
duplicateRows = df_deals[df_deals.duplicated()]

In [12]:
duplicateRows

,1_post_date,2_post_title,3_amount,4_Stake,5_funding_round,6_investors,7_source,Country/Town,Country(HQ),Category,main_sector,Check vs Companies,"Deals Information, Level of Completeness",duplicated_conc,Disclosed,Founded,Total Disclosed Funding,DA Classification_African Company (Yes = 0; No = 1),"funding round, DA","Industry, DA",Year,Month,Quarter,Half,Number of Investors,Investor 1,Investor 2,Investor 3,Investor 4,Investor 5,Investor 6,Investor 7,Investor 8,Investor 9,Investor 10,Investor 11,Investor 12,Investor 13,Investor 14,Cummulative Deals (Disclosed),Cummulative Deals (Undisclosed),Cummulative Amount,Investor 1.1,Investor 2.1,Investor 3.1,Investor 4.1,Investor 5.1,Investor 6.1,Investor 7.1,Investor 8.1,Investor 9.1,Investor 10.1,Investor 11.1,Investor 12.1
489,2018-04-19,Shezlong,350000,NaN,Seed,"500 Startups, Endure Capital, HIMangel",https://www.menabytes.com/shezlong-350k/,Egypt,Egypt,"Health Care, Medical, mHealth",Healthcare & Pharma,Shezlong,0.900,43209Shezlong350000,1.000,2015.000,950000.000,0.000,2. Seed,Health & Pharmaceuticals,2018.000,4.000,2.000,1.000,3.000,Seedstars,Endure Capital,HIMangel,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Seedstars,Endure Capital,HIMangel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
558,2018-06-05,Onesha,50000,NaN,Seed,Pangea Accelerator,http://disrupt-africa.com/2018/06/pangea-inves...,Kenya,Kenya,"Architecture, Brand Marketing, Digital Marketi...",Media & Entertainment,Onesha,0.900,43256Onesha50000,1.000,2016.000,100000.000,0.000,2. Seed,Media & Entertainment,2018.000,6.000,2.000,1.000,1.000,Seedstars,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Seedstars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900.000,1.000,1.000,1.000,1.000,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1280,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900.000,1.000,1.000,1.000,1.000,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1281,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900.000,1.000,1.000,1.000,1.000,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2055,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2056,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2057,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_deals

,1_post_date,2_post_title,3_amount,4_Stake,5_funding_round,6_investors,7_source,Country/Town,Country(HQ),Category,main_sector,Check vs Companies,"Deals Information, Level of Completeness",duplicated_conc,Disclosed,Founded,Total Disclosed Funding,DA Classification_African Company (Yes = 0; No = 1),"funding round, DA","Industry, DA",Year,Month,Quarter,Half,Number of Investors,Investor 1,Investor 2,Investor 3,Investor 4,Investor 5,Investor 6,Investor 7,Investor 8,Investor 9,Investor 10,Investor 11,Investor 12,Investor 13,Investor 14,Cummulative Deals (Disclosed),Cummulative Deals (Undisclosed),Cummulative Amount,Investor 1.1,Investor 2.1,Investor 3.1,Investor 4.1,Investor 5.1,Investor 6.1,Investor 7.1,Investor 8.1,Investor 9.1,Investor 10.1,Investor 11.1,Investor 12.1
0,2008-09-01,biNu,600000,NaN,Seed,Artesian VC,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000,9.000,3.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-11-01,biNu,400000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,Australia,Australia,Mobile Internet,Information Technology,biNu,0.900,NaN,1.000,2008.000,14220000.000,1.000,2. Seed,Other Technologies & Information Technology,2008.000,11.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-12-03,AllLife,6000000,NaN,Private Equity,LeapFrog Investments,http://www.prnewswire.co.uk/news-releases/leap...,South Africa,South Africa,"Insurance, InsurTech",Financial Services,AllLife,0.900,NaN,1.000,2004.000,12700000.000,0.000,6. Private Equity,Financial Services,2009.000,12.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-12-11,Bridge International Academies,1800000,NaN,Grant,Omidyar Network,http://foundationcenter.org/pnd/news/story.jht...,NaN,NaN,NaN,NaN,NaN,0.643,NaN,1.000,NaN,27800000.000,NaN,1. Grant,NaN,2009.000,12.000,4.000,2.000,1.000,Omidyar Network,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Omidyar Network,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-01,biNu,320000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.000,4.000,2.000,1.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2055,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2056,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2057,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Data cleaning

In [14]:
# Unique_deals

In [15]:
df_deals['1_post_date'] = df_deals['1_post_date'].astype('datetime64')

In [16]:
#filling nan values with mode of the year column
df_deals['1_post_date'] = df_deals['1_post_date'].fillna(df_deals['1_post_date'].mode()[0])

In [17]:
#Add year column from 1_post_date
def get_year(dt):
    return dt.year

df_deals['year'] = df_deals['1_post_date'].map(get_year)
#df_deals

In [18]:
#drop duplicate rows unique deals
Unique_deals_df = df_deals.drop_duplicates()

In [19]:
year_with_highest_deals_df = Unique_deals_df[['year','2_post_title']]

In [20]:
yrVposts = Unique_deals_df.year.value_counts()
yrVposts

2018    510
2019    366
2017    153
2016    108
2015     60
2014     31
2013     20
2012     16
2011      6
2010      4
2008      2
2009      2
2007      1
Name: year, dtype: int64

In [21]:
yrVposts.to_csv('yearVposts.json',index = False)

value of investment yearly, monthly quartly

In [22]:
Unique_deals_df

,1_post_date,2_post_title,3_amount,4_Stake,5_funding_round,6_investors,7_source,Country/Town,Country(HQ),Category,main_sector,Check vs Companies,"Deals Information, Level of Completeness",duplicated_conc,Disclosed,Founded,Total Disclosed Funding,DA Classification_African Company (Yes = 0; No = 1),"funding round, DA","Industry, DA",Year,Month,Quarter,Half,Number of Investors,Investor 1,Investor 2,Investor 3,Investor 4,Investor 5,Investor 6,Investor 7,Investor 8,Investor 9,Investor 10,Investor 11,Investor 12,Investor 13,Investor 14,Cummulative Deals (Disclosed),Cummulative Deals (Undisclosed),Cummulative Amount,Investor 1.1,Investor 2.1,Investor 3.1,Investor 4.1,Investor 5.1,Investor 6.1,Investor 7.1,Investor 8.1,Investor 9.1,Investor 10.1,Investor 11.1,Investor 12.1,year
0,2008-09-01,biNu,600000,NaN,Seed,Artesian VC,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000,9.000,3.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
1,2008-11-01,biNu,400000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,Australia,Australia,Mobile Internet,Information Technology,biNu,0.900,NaN,1.000,2008.000,14220000.000,1.000,2. Seed,Other Technologies & Information Technology,2008.000,11.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008
2,2009-12-03,AllLife,6000000,NaN,Private Equity,LeapFrog Investments,http://www.prnewswire.co.uk/news-releases/leap...,South Africa,South Africa,"Insurance, InsurTech",Financial Services,AllLife,0.900,NaN,1.000,2004.000,12700000.000,0.000,6. Private Equity,Financial Services,2009.000,12.000,4.000,2.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
3,2009-12-11,Bridge International Academies,1800000,NaN,Grant,Omidyar Network,http://foundationcenter.org/pnd/news/story.jht...,NaN,NaN,NaN,NaN,NaN,0.643,NaN,1.000,NaN,27800000.000,NaN,1. Grant,NaN,2009.000,12.000,4.000,2.000,1.000,Omidyar Network,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Omidyar Network,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
4,2010-04-01,biNu,320000,NaN,Angel,Undisclosed,https://www.crunchbase.com/search/funding_roun...,NaN,NaN,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.000,4.000,2.000,1.000,1.000,#REF!,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,#REF!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1276,2019-09-18,Sesi Technologies,50000,NaN,Grant,GoGettaz Agripreneur Prize,https://disrupt-africa.com/2019/09/botswanan-g...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.000,9.000,3.000,2.000,1.000,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1277,2019-09-18,Maungo Craft,50000,NaN,Grant,GoGettaz Agripreneur Prize,https://disrupt-africa.com/2019/09/botswanan-g...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.000,9.000,3.000,2.000,1.000,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1278,2019-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900.000,1.000,1.000,1.000,1.000,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1304,2019-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900.000,1.000,NaN,NaN,NaN,Johnson & Johnson,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019


In [23]:
df_deals['3_amount'].isna().sum()
df_deals['3_amount']

781

0        600000
1        400000
2       6000000
3       1800000
4        320000
         ...   
2054        NaN
2055        NaN
2056        NaN
2057        NaN
2058        NaN
Name: 3_amount, Length: 2059, dtype: object

In [24]:
# Filling all nan values with zero
df_deals['3_amount'] = df_deals['3_amount'].fillna(0)

In [25]:
# df_deals['3_amount'].astype('int')
df_deals['3_amount'] = df_deals['3_amount'].replace('Undisclosed', '0')
df_deals['3_amount'] = df_deals['3_amount'].astype(int)


In [26]:
print(df_deals.dtypes)

1_post_date        datetime64[ns]
2_post_title               object
3_amount                    int64
4_Stake                   float64
5_funding_round            object
                        ...      
Investor 9.1               object
Investor 10.1              object
Investor 11.1              object
Investor 12.1              object
year                        int64
Length: 55, dtype: object


In [27]:
df_deals['3_amount'].astype('int')
df_deals['3_amount'].unique()

0        600000
1        400000
2       6000000
3       1800000
4        320000
         ...   
2054          0
2055          0
2056          0
2057          0
2058          0
Name: 3_amount, Length: 2059, dtype: int64

array([    600000,     400000,    6000000,    1800000,     320000,
            75000,     700000,     150000,    1500000,          0,
           865000,     864900,    3000000,   45000000,    3500000,
          4000000,     500000,      20000,    9000000,    2000000,
            10000,   23000000,    4300000,  150000000,   10000000,
           240000,   20000000,    1100000,     120000,     175000,
          2600000,      25000,     300000,     169000,    8000000,
         25000000,     715000,   16000000,       8400,    5500000,
         11000000,    7000000,     750000,     250000,     938000,
          2700000,  100000000,     650000,     115000,    1700000,
          1200000,     450000,    1400000,      50000,      80000,
           330000,     200000,   50000000,    2260000,     780000,
         12500000,    5000000,   41000000,     680000,   14000000,
           142000,    1000000,     900000,     180000,     100000,
          2400000,   13000000,    2100000,     355000,   19000

In [28]:
Unique_deals_df['year']

0       2008
1       2008
2       2009
3       2009
4       2010
        ... 
1276    2019
1277    2019
1278    2019
1304    2019
1533    2019
Name: year, Length: 1279, dtype: int64

In [29]:
print(Unique_deals_df.dtypes)
Unique_deals_df['3_amount'].astype(int)

1_post_date        datetime64[ns]
2_post_title               object
3_amount                   object
4_Stake                   float64
5_funding_round            object
                        ...      
Investor 9.1               object
Investor 10.1              object
Investor 11.1              object
Investor 12.1              object
year                        int64
Length: 55, dtype: object


ValueError: invalid literal for int() with base 10: 'Undisclosed'

In [ ]:
Unique_deals_df['year'].unique()

In [ ]:
value_of_investment_yearly = Unique_deals_df.groupby('year')['3_amount'].sum()

In [ ]:
value_of_investment_yearly

In [ ]:
# value_of_investment_yearly.plot(kind='bar')